In [3]:
#!pip install byol-pytorch
#!pip install torchvision

You should consider upgrading via the 'c:\users\mr.mohebbian\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [1]:
import torch
import torch.nn as nn
from byol_pytorch import BYOL
from torchvision import models
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
resnet = models.resnet50(pretrained=True)


In [2]:
import requests
requests.packages.urllib3.disable_warnings()

import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

In [2]:
learner = BYOL(
    resnet,
    image_size = 256,
    projection_size = 10,           # the projection size
    projection_hidden_size = 1024,   # the hidden dimension of the MLP for both the projection and prediction
    moving_average_decay = 0.99,      # the moving average decay factor for the target encoder, already set at what paper recommends
    hidden_layer = 'avgpool'
)

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True,  transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=2,
                                          shuffle=True, num_workers=1)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True,  transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=2,
                                         shuffle=False, num_workers=1)
opt = torch.optim.Adam(learner.parameters(), lr=3e-4)


C:\Users\MR.Mohebbian\AppData\Local\Programs\Python\Python37\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
correct = 0
total = 0
with torch.no_grad():
    for i, data in enumerate(testloader):
        images, labels = data
        outputs = resnet(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if i==100:
            break

print('Accuracy of the network on the 100 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 100 test images: 0 %


In [5]:
for i, data in enumerate(trainloader, 0):
    images, labels = data
    loss = learner(images)
    opt.zero_grad()
    loss.backward()
    opt.step()
    learner.update_moving_average() # update moving average of target encoder
    print(f"iteration : {i}-- loss: {loss.item()}")
    if i==20:
        break

# save your improved network
torch.save(resnet.state_dict(), './improved-net.pt')

iteration : 0-- loss: 1.8331477642059326
iteration : 1-- loss: 2.0722947120666504
iteration : 2-- loss: 2.332798719406128
iteration : 3-- loss: 1.8694794178009033
iteration : 4-- loss: 1.8785459995269775
iteration : 5-- loss: 1.9752558469772339
iteration : 6-- loss: 1.4972285032272339
iteration : 7-- loss: 1.9930981397628784
iteration : 8-- loss: 1.999084234237671
iteration : 9-- loss: 2.2616498470306396
iteration : 10-- loss: 2.1406755447387695
iteration : 11-- loss: 1.6278018951416016
iteration : 12-- loss: 2.3324742317199707
iteration : 13-- loss: 2.229515552520752
iteration : 14-- loss: 1.8463784456253052
iteration : 15-- loss: 2.3014895915985107
iteration : 16-- loss: 2.038620948791504
iteration : 17-- loss: 2.5905003547668457
iteration : 18-- loss: 2.3642516136169434
iteration : 19-- loss: 1.2921377420425415
iteration : 20-- loss: 2.0660958290100098


In [6]:
correct = 0
total = 0
with torch.no_grad():
    for i, data in enumerate(testloader):
        images, labels = data
        projection, embedding = learner(images, return_embedding = True)
        _, predicted = torch.max(projection.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if i==100:
            break

print('Accuracy of the network on the 1000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 1000 test images: 10 %
